In [1]:
import os, shutil
import numpy as np

from matplotlib import image, pyplot
from skimage.transform import resize


from tensorflow.keras.utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Conv2D,MaxPool2D,Dense,Flatten,Dropout

from keras import callbacks
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,accuracy_score, precision_score, f1_score, recall_score,classification_report,roc_curve, auc 
from sklearn.utils import class_weight

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn import svm
from PIL import Image,ImageFile
from tensorflow.keras import models
from tensorflow.keras import Model
from tensorflow.keras import layers
from tensorflow.keras import optimizers

In [2]:
resultPath = 'C:/Users/HP/Downloads/Results/CNN'

In [3]:
DATA_PATH = "C:/Users/HP/Downloads/Image_Hunter_dataset/"
spamData=[]
label=[]
hashList=[]
count=-1
for folder in os.listdir(DATA_PATH):
    print(">>>Reading ",folder)
    count+=1
    
    for file in os.listdir(DATA_PATH+folder):
        if(str(file).endswith('.jpg') or str(file).endswith('.JPG') or str(file).endswith('.jpeg') or str(file).endswith('.JPEG')):
            img = image.imread(DATA_PATH+folder+'/'+file)
            hsh = hash(tuple(np.array(img).flatten()))
            if(hsh not in hashList):
                spamData.append(resize(img, (156, 156, 3)))
                hashList.append(hsh)
                label.append(count)
spamData=np.array(spamData)
label=np.array(label)

>>>Reading  NaturalImages
>>>Reading  SpamImages


In [4]:
print("Number of SPAM",len(label[label==0]))
print("Number of HAM",len(label[label==1]))

Number of SPAM 810
Number of HAM 895


In [5]:
print("Spam data shape : ",spamData.shape," Label shape : ",label.shape)

Spam data shape :  (1705, 156, 156, 3)  Label shape :  (1705,)


In [6]:
test_valid_Percentage = 0.3
x_train,x_test,y_train,y_test = train_test_split(spamData,label,test_size = test_valid_Percentage,random_state=42, stratify=label,shuffle=True)

In [7]:
print("x_train shape : ",x_train.shape," y_train shape : ",y_train.shape)
print("x_test shape : ",x_test.shape," y_test shape : ",y_test.shape)

x_train shape :  (1193, 156, 156, 3)  y_train shape :  (1193,)
x_test shape :  (512, 156, 156, 3)  y_test shape :  (512,)


In [8]:
print("Number of train SPAM",len(y_train[y_train==0]))
print("Number of train HAM",len(y_train[y_train==1]))

print("Number of test SPAM",len(y_test[y_test==0]))
print("Number of test HAM",len(y_test[y_test==1]))

Number of train SPAM 567
Number of train HAM 626
Number of test SPAM 243
Number of test HAM 269


In [9]:
y_train_oh = to_categorical(y_train)
y_test_oh = to_categorical(y_test)

print("y_train_oh shape : ",y_train_oh.shape," y_test_oh shape : ",y_test_oh.shape)

y_train_oh shape :  (1193, 2)  y_test_oh shape :  (512, 2)


In [13]:
#Initializing Exception Network as Base Model for Transfer Learning

from tensorflow.keras.applications.vgg19 import VGG19

base_model_vgg = VGG19(input_shape=(156, 156,3), weights='imagenet', include_top=False,classes= 2)

for layer in base_model_vgg.layers[:16]:
    layer.trainable = False
for layer in base_model_vgg.layers[17:]:
    layer.trainable = True

for layer in base_model_vgg.layers:
    print(layer.name , layer.trainable)
len(base_model_vgg.layers)

input_4 False
block1_conv1 False
block1_conv2 False
block1_pool False
block2_conv1 False
block2_conv2 False
block2_pool False
block3_conv1 False
block3_conv2 False
block3_conv3 False
block3_conv4 False
block3_pool False
block4_conv1 False
block4_conv2 False
block4_conv3 False
block4_conv4 False
block4_pool True
block5_conv1 True
block5_conv2 True
block5_conv3 True
block5_conv4 True
block5_pool True


22

In [15]:
NO_OF_EPOCHS=10
BATCH_SIZE=64

model=Sequential()
model.add(base_model_vgg)
model.add(Flatten())
model.add(Dense(128,activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1,activation='sigmoid'))
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg19 (Functional)          (None, 4, 4, 512)         20024384  
                                                                 
 flatten (Flatten)           (None, 8192)              0         
                                                                 
 dense (Dense)               (None, 128)               1048704   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                                 
Total params: 21,073,217
Trainable params: 10,488,065
Non-trainable params: 10,585,152
_________________________________________________________________


In [16]:
checkpointer = callbacks.ModelCheckpoint(filepath=resultPath+"/checkpoint-{epoch:04d}.hdf5", verbose=1, save_best_only=True, monitor='val_acc',mode='max')
csv_logger = CSVLogger(resultPath+'/result_logger.csv',separator=',', append=False)
reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.2, patience=2, min_lr=0.001)
model.fit(x_train,y_train,epochs= 10,verbose=1,batch_size= 64,validation_data=(x_test,y_test),callbacks=[checkpointer,csv_logger,reduce_lr])

Epoch 1/10
19/19 [==============================] - 145s 7s/step - loss: 0.9264 - accuracy: 0.5507 - val_loss: 0.5476 - val_accuracy: 0.6523 - lr: 0.0010
Epoch 2/10
19/19 [==============================] - 146s 8s/step - loss: 0.1914 - accuracy: 0.9455 - val_loss: 0.0875 - val_accuracy: 0.9883 - lr: 0.0010
Epoch 3/10
19/19 [==============================] - 148s 8s/step - loss: 0.0976 - accuracy: 0.9782 - val_loss: 0.0679 - val_accuracy: 0.9863 - lr: 0.0010
Epoch 4/10
19/19 [==============================] - 143s 8s/step - loss: 0.0655 - accuracy: 0.9832 - val_loss: 0.0581 - val_accuracy: 0.9863 - lr: 0.0010
Epoch 5/10
19/19 [==============================] - 142s 8s/step - loss: 0.0406 - accuracy: 0.9866 - val_loss: 0.5905 - val_accuracy: 0.9199 - lr: 0.0010
Epoch 6/10
19/19 [==============================] - 143s 8s/step - loss: 0.1744 - accuracy: 0.9749 - val_loss: 0.2590 - val_accuracy: 0.9863 - lr: 0.0010
Epoch 7/10
19/19 [==============================] - 147s 8s/step - loss: 0.0

In [17]:
prediction_prob1 = model.predict(x_test,verbose=1)
y_pred=np.round(prediction_prob1)

16/16 [==============================] - 36s 2s/step


In [18]:
print(accuracy_score(y_test,y_pred))
print(precision_score(y_test,y_pred,average='binary'))
print(recall_score(y_test,y_pred,average='binary'))
print(f1_score(y_test,y_pred,average='binary'))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

0.98828125
0.9888475836431226
0.9888475836431226
0.9888475836431226
[[240   3]
 [  3 266]]
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       243
           1       0.99      0.99      0.99       269

    accuracy                           0.99       512
   macro avg       0.99      0.99      0.99       512
weighted avg       0.99      0.99      0.99       512

